### Portail meteo.data.gouv.fr - Téléchargement-affichage-Extraction des données QUOTIDIENNES LATEST de Météo-France (RR-T-Vent)
Ce notebook traite uniquement les fichiers "RR-T-vent" (Paramètres liés aux Précipitations, Température & Vent) à l'exclusion des fichiers "autres-parametres" contenant notamment l'ETP.<br>
Une connexion internet est nécessaire pour le téléchargement automatique des archives de données.

1) Téléchargement des fichiers quotidiens et décompression automatique, pour différents postes dans plusieurs départements si besoin
2) Tracé du graphique chronologique QUOTIDIEN pour le paramètre Précipitations RR des postes choisis par l'utilisateur
3) Tracé du graphique chronologique de l'agrégation MENSUELLE
4) Sauvegarde d'un fichier excel rassemblant:
    - Les données intégrales QUOTIDIENNES et agrégées MENSUELLEMENT (tous paramètres de la période "Latest" pour tous les postes des départements concernés).
    - la comparaison des paramètres pour les postes et la période choisis, ainsi que pour les précipitations les graphiques quotidien & mensuel

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/6a8df7e9-45ff-445d-9260-6c65475dda86

Utilisez mon autre script pour visualiser la carte des postes météorologiques dont la liste est fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

NB: 
- Les données LATEST correspondent aux DERNIERS FICHIERS mis à jour quotidiennement, et qui vont du mois de janvier de l'année précédente à la veille du jour en cours même partielle.
- Les données QUOTDIENNES agrégées mensuellement ne sont pas simplement équivalentes aux donnée MENSUELLES. Les paramètres ne sont pas strictement les mêmes. Certains paramètres quotidiens n'existent pas en mensuel (ex. FF2M vent à 2 m, HTN heure de Tmin), mais les paramètres mensuels sont globalement plus nombreux à cause des paramètres "calculés" (par exemple des NOMBRES DE JOURS DE PLUIE/GELEE/CHALEUR/etc.., ainsi que des précipitations MENSUELLES ESTIMEES 'RR_ME' plus anciennes que les mesures)

Auteur: https://github.com/loicduffar

##### 1) Lecture après téléchargement des archives GZ et décompression des fichiers CSV LATEST (dernière période disponible)

- Définir les chemins d'enregistrement pour les archives GZ et pour les fichier CSV décompressés
- Définir les postes à interroger par leur code à 8 chiffres (voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922)
- Définir UNE FOIS POUR TOUTES les urls de téléchargement du fichier LATEST pour chaque département
- NB: Les paramètres et la période à extraire sont définis dans la cellule 2 (pour pouvoir les modifier sans relire les fichiers)
- On peut choisir d'éviter de télécharger-décompresser les archives GZ si le dernier téléchargement date du jour même

Les archives LATEST sont automatiquement téléchargées et décompressées et les fichiers CSV sont lus

In [1]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Télécharge les archives GZ & et décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE pour chaque département)
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\Q"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q"

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS (voir plus loin ======= Initialisation ==========)
postes= [
        '04088001', '04039001', '04230001',                  # Forcalquier, Castellane, Valensole \
        # '05046001',                                           # Embrun \
        # '06088001', '06029001',                               # Nice, Cannes \
        '13001009', '13111002', '13103001','13055001',        # Aix en Provence, Vauvenargue, Salon, Marseille, \
        '83031001', '83061001', '83137001',                   # Le Luc, Fréjus, Toulon \
        '84003002', '84009002',                               # # Apt-Viton, Bastide des Jourdans \
        # '97411132', '97411150',                               # 974 LA REUNION: Chaudron, Saint-François \
        # '98511001',                                            # 985 MAYOTTE: Mamoudzou_SAPC \
        # '98404001', '98404002',                                 # 984 TERRES AUSTRALES ET ANTARTIQUES FRANCAISES: Kerguelen, Nouvelle Amsterdam \        
        ]                               

# Choisir d'éviter le téléchargement/décompression
# non recommandé, sauf pour test ou à moins que le dernier téléchargement date du jour même
download= False

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMMJJ	RR	QRR	TN	QTN	HTN	QHTN	TX	QTX	HTX	QHTX	TM	QTM	TNTXM	QTNTXM	TAMPLI	QTAMPLI
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220101	0	1	4.6	1	820	9	9.4	1	1424	9	7.9	1	7	1	4.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220102	0	1	8.3	1	646	9	11.1	1	1145	9	9.1	1	9.7	1	2.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220103	0.2	1	4.6	1	655	9	13.5	1	1452	9	7.4	1	9.1	1	8.9	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220104	1.4	1	4.5	1	2348	9	13.7	1	1123	9	10.1	1	9.1	1	9.2	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	20220105	0	1	4.6	1	1742	9	8.7	1	601	9	6.8	1	6.7	1	4.1	1

# ================ Extrait du fichier CSV Descriptif de quelques paramètres (les plus souvent utiles, précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJ    	 date de la mesure (année mois jour)
# RR          	 quantité de précipitation tombée en 24 heures (de 06h FU le jour J à 06h FU le jour J+1). La valeur relevée à J+1 est affectée au jour J (en mm et 1/10)
# TN          	 température minimale sous abri (en °C et 1/10)
# HTN         	 heure de TN (hhmm)
# TX          	 température maximale sous abri (en °C et 1/10)
# HTX         	 heure de TX (hhmm)
# TM          	 moyenne quotidienne des températures horaires sous abri (en °C et 1/10)
# TNTXM       	 moyenne quotidienne (TN+TX)/2 (en °C et 1/10)
# TAMPLI      	 amplitude thermique quotidienne -  écart entre TX et TN quotidiens (TX-TN) (en °C et 1/10)
# TNSOL       	 température quotidienne minimale à 10 cm au-dessus du sol (en °C et 1/10)
# TN50        	 température quotidienne minimale à 50 cm au-dessus du sol (en °C et 1/10)
# DG          	 durée de gel sous abri (T ≤ 0°C) (en mn)

# ================ Initialisation ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))
# Structure du nom des fichiers de données QUOTIDIENNES (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_start= 'Q_' # début du nom du fichier
# Fin du nom dépendant de l'année en cours
template_end= '_latest-' + str(now.year-1) + '-' + str(now.year) + '_RR-T-Vent.csv'

# ---------------- urls de téléchargement des archives PAR DEPARTEMENT des dernières données depuis janvier de l'année précédente https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
url_desc_q= "https://www.data.gouv.fr/fr/datasets/r/6a8df7e9-45ff-445d-9260-6c65475dda86"
url_desc_m= "https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e"
urls= dict()
urls['04']= "https://www.data.gouv.fr/fr/datasets/r/52355223-df85-49a9-9031-14b8a604c28c" 
urls['05']= 'https://www.data.gouv.fr/fr/datasets/r/27bfba71-f9d2-4cbb-a784-7a893d933485'
urls['06']= 'https://www.data.gouv.fr/fr/datasets/r/6d848c25-210b-40a4-b3c2-a4084bf660a7'
urls['13']= 'https://www.data.gouv.fr/fr/datasets/r/eb4d0600-e90b-4517-a429-599ed13dbae0'
urls['83']= 'https://www.data.gouv.fr/fr/datasets/r/5889ea7c-e285-46cb-8ea6-1c317fa4dada'
urls['84']= 'https://www.data.gouv.fr/fr/datasets/r/b47bd200-0ba7-419c-9385-4ce8ff8c199c'
urls['974']= 'https://www.data.gouv.fr/fr/datasets/r/50a0d0fb-d97c-4ca3-b308-def80d69416d' # La Réunion
urls['975']= 'https://www.data.gouv.fr/fr/datasets/r/86b3a033-b7be-4733-9203-cef708aad6a4' # Saint-Pierre et Miquelon
urls['984']= 'https://www.data.gouv.fr/fr/datasets/r/3a82cde3-63aa-4b57-b8ba-84820e92c910' # Terres Australes et Antarctiques Françaises
urls['985']= 'https://www.data.gouv.fr/fr/datasets/r/8c5daf91-4634-40d8-822d-27f1c04cf2b4' # Mayotte

# ---------------- Définition des fonctions
def convert_to_date(chaine):
    return pd.to_datetime(str(chaine), format='%Y%m%d', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMMJJ':str}, parse_dates=['AAAAMMJJ'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")
    return df

# ================ Traitement ====================
# Lecture en ligne du fichier JSON
req= requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script
df_liste_postes = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df_liste_postes['lat'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df_liste_postes['lon'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df_liste_postes['nom_usuel']= df_liste_postes['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df_liste_postes['num_poste']= df_liste_postes['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df_liste_postes['commune'] = df_liste_postes['properties'].apply(lambda x: x['COMMUNE'].strip())
df_liste_postes['ficheClimComplete'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df_liste_postes['ficheClimReduite'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df_liste_postes['alti'] = df_liste_postes['properties'].apply(lambda x: x['ALTI'])
# supprime les colonnes inutiles et classe par numéro de poste (donc par département)
df_liste_postes.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df_liste_postes.sort_values(by= ['num_poste'], inplace=True)

# Lit en ligne le fichier "fin" "Q_descriptif_champs_RR-T-Vent.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_f = pd.read_csv(url_desc_q, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_f.index= df_desc_f.index.str.strip()
df_desc_f['name_long']= df_desc_f['name_long'].str.strip()
# Lit en ligne le fichier "MENSQ_descriptif_champs.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_bis = pd.read_csv(url_desc_m, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_bis.index= df_desc_bis.index.str.strip()
df_desc_bis['name_long']= df_desc_bis['name_long'].str.strip()

# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
departements = []
for poste in postes:
    departements.append(poste[:2])
# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
i, j= 0, 0
for departement in departements:
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    if download:
        download_file(url, filename)
        decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    df_departement= read_csv(filename)
    if i== 0: # pour le premier département, initialisation du dataframe final df        
        df= df_departement
        i= i+1
    else: # sinon concaténation du nouveau département dans le dataframe df
        df= pd.concat([df, df_departement])        

# supprime les espaces avant et après le numéro de poste
df['NUM_POSTE'] = df['NUM_POSTE'].str.strip()
# renome la colonne "AAAMMJJ" en "DATE"
df.rename(columns={'AAAAMMJJ':'DATE'}, inplace=True)
# supprime les espaces dans les noms de colonnes
df.columns= df.columns.str.strip()

display(df)

# Détection des postes manquants dans la liste JSON
postes_mesures= df['NUM_POSTE'].unique()
postes_json= df_liste_postes['num_poste'].unique()
# trouvrer les postes_mesures absents de la liste JSON
postes_absents= np.setdiff1d(postes_mesures, postes_json)
if len(postes_absents) > 0:
    # trouver le NOM_USEL des postes absents
    print("Postes absents de la liste JSON:", postes_absents, df[df['NUM_POSTE'].isin(postes_absents)]['NOM_USUEL'].unique())
else:
    print("Aucun postes absent de la liste JSON")

2023-12-31 13:43
Départements concernés:  ['04', '13', '83', '84']
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_04_latest-2022-2023_RR-T-Vent.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_13_latest-2022-2023_RR-T-Vent.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_83_latest-2022-2023_RR-T-Vent.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\Q_84_latest-2022-2023_RR-T-Vent.csv


,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,DATE,RR,QRR,TN,QTN,...,DXI2,QDXI2,HXI2,QHXI2,FXI3S,QFXI3S,DXI3S,QDXI3S,HXI3S,QHXI3S
0,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-01,0.0,1.0,1.1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-02,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-03,0.0,1.0,-0.4,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-04,1.6,1.0,0.2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-05,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12937,84150001,VISAN,44.336667,4.905500,141,2023-12-16,0.0,1.0,5.4,1.0,...,NaN,NaN,NaN,NaN,15.0,1.0,NaN,NaN,728.0,9.0
12938,84150001,VISAN,44.336667,4.905500,141,2023-12-17,0.0,1.0,-1.7,1.0,...,NaN,NaN,NaN,NaN,9.1,1.0,NaN,NaN,37.0,9.0
12939,84150001,VISAN,44.336667,4.905500,141,2023-12-18,0.2,1.0,-3.4,1.0,...,NaN,NaN,NaN,NaN,3.1,1.0,NaN,NaN,1258.0,9.0
12940,84150001,VISAN,44.336667,4.905500,141,2023-12-19,0.2,1.0,-2.6,1.0,...,NaN,NaN,NaN,NaN,12.7,1.0,NaN,NaN,2319.0,9.0


Postes absents de la liste JSON: ['04006400' '04019404' '04088001' '04112010' '04205001' '04219001'
 '04226001'] ['La Foux d Allos' 'Pra Loup' 'FORCALQUIER' 'MANOSQUE-PIMARLET' 'SEYNE'
 'THORAME HAUTE' 'UVERNET FOURS']


##### 2) Affichage des données quotidiennes & Graphique des précipitations 

- Définir comme suit la période d'extraction : (le graphique ne commence qu'au début du mois précédent, mais on peut l'étendre à la souris à l'intérieur de la période d'extraction)
    - None pour extraire les données depuis le 1er septembre de l'avant-dernière année de l'archive. La période d'extraction conditionne également le début de la courbe de pluie cumulée  
    - ou définir la date de début et de fin d'extraction, ce qui permet d'étendre l'extraction si besoin jusqu'au début de l'archive
- Définir les paramètres à extraire (au minimum les précipitations RR car le graphique est uniquement prévu pour cela)
- NB: Tous les paramètres sont des chiffres, y compris ceux donnant une heure (ex. HTN= 738 -> Heure du minimum de température= 07:38)

In [10]:
# trace un graphique plotly de df avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import calendar
import numpy as np

#-------------- Personalisation du graphique
# Paramètres à extraire: 'RR' obligatoire en premier, pour le graphique dont la présentation actuelle est spécialement adaptée aux précipitations
param_list= ['RR', 'TM']
agregat_list= ['sum', 'mean'] # 'sum' pour la somme (précipitations), 'mean' pour la moyenne (température moyenne), max (vent), min (Tempéraure min)
# TODO: prenrdre en compte le Nombre maxi de NaN dans les données pour l'agrégation mensuelle
# agregat_max_nan_list= [0, 5]

# Période A EXTRAIRE (None pour la période par défaut)
start_date= datetime.datetime(2023, 9, 1)
start_date= None

end_date= datetime.datetime(2023, 8, 1)
end_date= None

#-------------- initialisation des paramètres du graphique
param_RR= 'RR'
# Période par défaut d'extraction (si start_date= None ou  end_date= None): maximisation de l'extraction
if end_date is None:
    end_date= df['DATE'].max()
if start_date is None:
    if end_date.month >= 9:
        start_date= datetime.datetime(end_date.year, 9, 1)
    else:
        start_date= datetime.datetime(end_date.year - 1, 9, 1) # 1er septembre de l'année précédente

# période du GRAPHIQUE : premier jour du premier mois au début du mois suivant (pour une graduation clair de l'axe des dates)
end_graph= end_date
start_graph = datetime.datetime(end_graph.year, end_graph.month, 1) - datetime.timedelta(days= 31) # calcule start_graph  comme le premier jour du mois précédant end_date

# filtre le dataframe sur les postes choisis
df_postes= df[df['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Dark24
color_sequence_scatter = plotly.colors.qualitative.Alphabet

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param_RR], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)],  # Utilisez la couleur correspondante de la séquence
                                    line= dict(color='rgb(100,100,100)', width=1),  # Ligne de contour des barres
                                    ),
                        # hovertemplate = 
                            # '<i>Date</i>: %{x}' +
                            # '<br><b>RR</b>: %{y}<br>',
                            # text = df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]                                    
                         ),
                row= i+1, col= 1
                )
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param_RR].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]
                                        ), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                    # hovertemplate = 
                    #         '<br><b>Cumulé</b>: %{y}<br>'                                      
                             ), 
                 row=i+1, col=1, secondary_y= True,
                 )

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations quotidiennes (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                    )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot (empêche la graduation des dates de tous les subplots)

# Définition générale des axes
max_value= df_postes[param_RR].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()[param_RR].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# GRADUATION de l'axe des DATES ........................................
if end_graph - start_graph > datetime.timedelta(days= 365): # plus d'un an
    tickformat= '%d-%b\n%Y'
    dtick= 'M2'
else: # Moins
    tickformat= '%d %b\n%Y'
    dtick= 'M1' # (pour une raison inconnue,'D10' ne marche pas du tout et le nombre de milisecondes ne marche qu'après la 2ème graduation et )
# ............................... Premiers subplots
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ............................... Dernier subplot
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x', range=[start_graph, end_graph])

# GRILLE.................................................................
# .................. verticales 
# .............................. Premiers subplots INUTILE ?
for i in range(1, len(postes) + 1):
    fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1)
# .............................. Dernier subplot TOUS ?
fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1)
# .................. horizontales
# ............................... Axe secondaire
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph= "meteo.data QUOT"
fig.write_html(os.path.join(folder_csv, file_graph + template_end[:-4] + ".html"))
fig.write_image(os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

#-------------- Données fines (quotidiennes) pivotée avec les postes en colonnes (pour la présentation ainsi que pour le calcul des agrégations mensuelles) 
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
for i, par in enumerate(param_list):
    if i==0:
        df_f = df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
            .assign(parametre= par)
    else:
        # append des lignes à df_f
        df_f = pd.concat([df_f,
                        df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
                        .assign(parametre= par)]
                        )
df_f= df_f.reset_index().set_index(['parametre', 'DATE'], drop= True)

#-------------- Agrégation mensuelle par cumul ou moyenne selon le paramètre, en affectant NaN aux mois incomplets
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
# TODO: préciser le test pour un nombre maxi de NaN dans les données agregat_max_nan_list
def sum_or_nan(s):
    return s.sum() if s.count() == s.size else np.nan
def mean_or_nan(s):
    return s.mean() if s.count() == s.size else np.nan
def max_or_nan(s):
    return s.max() if s.count() == s.size else np.nan
def min_or_nan(s):
    return s.min() if s.count() == s.size else np.nan

for i, par in enumerate(param_list):
    if i==0:
        if agregat_list[0] == 'sum':
            df_agr= df_f.loc[par].resample('MS').apply(sum_or_nan)
        elif agregat_list[0] == 'mean':
            df_agr= df_f.loc[par].resample('MS').apply(mean_or_nan)
        elif agregat_list[0] == 'max':
            df_agr= df_f.loc[par].resample('MS').apply(max_or_nan)
        df_agr= df_agr.reset_index().set_index([[par for j in range(df_agr.shape[0])], 'DATE'])
    else:
        if agregat_list[i] == 'sum':
            df_temp= df_f.loc[par].resample('MS').apply(sum_or_nan)
        elif agregat_list[i] == 'mean':
            df_temp= df_f.loc[par].resample('MS').apply(mean_or_nan)
        elif agregat_list[i] == 'max':
            df_temp= df_f.loc[par].resample('MS').apply(max_or_nan)
        df_agr= pd.concat([df_agr, 
                        df_temp.reset_index().set_index([[par for j in range(df_temp.shape[0])], 'DATE'])]
                        )
# arrondi df_agr à 1 chiffre après la virgule
df_agr= df_agr.round(1)

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# imprimer un avertissement si la dernière date du dataframe est antérieure au dernier jour du mois en cours
# calculer la date du dernier jour du mois en cours en tenant compte des jours à 28, 29 et 30 jours
last_date= datetime.datetime(now.year, now.month, calendar.monthrange(now.year, now.month)[1])
titre_agr= 'Agrégation mensuelle pour les paramètres et la période choisis: '
if np.datetime64(df['DATE'].values[-1]) < np.datetime64(last_date):
    titre_agr= titre_agr + 'Attention le cumul du dernier mois est incomplet à ce jour'
print(titre_agr)

for i, par in enumerate(param_list):
    # teste si df_desc_bis.loc[par] existe
    if par in df_desc_bis.index:
        titre= '- ' + par + ': ' + df_desc_bis.loc[par]['name_long']
    else:
        titre= '- ' + par + ': ' + 'agrégation mensuelle "' + agregat_list[i] + '" de "' + df_desc_f.loc[par]['name_long'] + '"'
    print(titre)
display(df_agr[(df_agr.index.get_level_values(1) >= start_graph) & (df_agr.index.get_level_values(1) <= end_graph)])

fig.show()

titre_f= 'Données quotidiennes pour les paramètres et la période choisis: '
print(titre_f)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_f.loc[par]['name_long']
    print(titre)
    display(df_f.loc[par][(df_f.loc[par].index >= start_date) & (df_f.loc[par].index <= end_date)])

2023-12-31 14:47
Agrégation mensuelle pour les paramètres et la période choisis: Attention le cumul du dernier mois est incomplet à ce jour
- RR: cumul mensuel des hauteurs de précipitation (en mm et 1/10)
- TM: moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)


NOM_USUEL      AIX EN PROVENCE  APT-VITON  CASTELLANE  FORCALQUIER  FREJUS  \
   DATE                                                                      
RR 2023-11-01             30.2       37.6       102.2         63.1    20.4   
   2023-12-01             42.6       60.4        76.3          NaN    50.6   
TM 2023-11-01              9.6        7.7         6.1          NaN    12.7   
   2023-12-01              7.0        5.4         2.6          NaN     9.4   

NOM_USUEL      LA BASTIDE DES JOURDANS  LE LUC  MARSEILLE-OBS  \
   DATE                                                         
RR 2023-11-01                     22.0    21.7           23.2   
   2023-12-01                     42.6    53.8           28.3   
TM 2023-11-01                      8.6    12.1           12.4   
   2023-12-01                      6.3     8.0            9.7   

NOM_USUEL      SALON DE PROVENCE  TOULON  VALENSOLE  VAUVENARGUES  
   DATE                                                            
RR 2023-11-01               30.8    25.8       36.4          41.8  
   2023-12-01               35.4    65.4       56.8          71.0  
TM 2023-11-01               10.2    13.4        8.4           8.5  
   2023-12-01                7.3    10.1        6.2           5.8

Données quotidiennes pour les paramètres et la période choisis: 
- RR: quantité de précipitation tombée en 24 heures (de 06h FU le jour J à 06h FU le jour J+1). La valeur relevée à J+1 est affectée au jour J (en mm et 1/10)


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2
2023-09-02,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0
2023-09-03,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-04,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-05,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-16,0.4,0.0,0.2,NaN,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
2023-12-17,0.0,0.0,0.2,NaN,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0
2023-12-18,0.2,0.0,0.2,NaN,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


- TM: moyenne quotidienne des températures horaires sous abri (en °C et 1/10)


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-09-01,22.1,21.0,17.5,NaN,21.8,19.7,21.8,22.5,21.2,21.3,19.8,19.1
2023-09-02,22.5,21.7,18.6,NaN,22.6,20.8,22.1,22.9,21.8,22.8,20.0,19.7
2023-09-03,24.3,23.0,19.2,NaN,23.2,23.0,23.3,24.6,23.5,23.7,22.5,21.9
2023-09-04,26.6,24.8,19.6,NaN,24.7,24.3,23.7,27.4,24.7,24.6,23.5,23.4
2023-09-05,25.2,23.1,18.8,NaN,23.4,22.3,23.6,26.2,23.9,25.4,22.2,22.2
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-16,5.1,1.6,-2.2,NaN,6.9,4.3,4.0,9.3,5.4,8.1,5.3,2.8
2023-12-17,5.4,3.0,-1.8,NaN,6.6,5.1,3.5,9.4,3.9,7.5,6.0,4.1
2023-12-18,6.2,4.6,-1.2,NaN,7.0,6.7,4.3,9.6,3.4,7.5,8.4,6.4


##### 3) Affichage de l'agrégation mensuelle & graphique des précipitations mensuelles

Définir au préalable comme suit les dates de début et de fin du graphique:
- "None" pour prendre le début et la fin du fichier LATEST (premier jour de l'année précédente et veille du téléchargement)
- ou définir la date de début et de fin

NB: Par défault les paramètres sont les mêmes que ceux choisis dans la cellule 2

In [11]:
# trace un graphique chronologique des précipitations mensuelles
# chaque poste de la colonne 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- Personalisation du graphique 
# Paramètre à représenter (la présentation actuelle est spécialement adaptée aux précipitations RR)
# param= 'RR'

# Période à représenter (None pour le début ou la fin du fichier par défaut)
# start_date= None
# start_date= datetime.datetime(2022, 5, 1)

# end_date= None
# end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
# à partir de df_postes, génère un dataframe indexé sur 'NOM_USUEL' avec une colonne 'NUM_POSTE' et
df_desc_postes= df_postes[['NUM_POSTE', 'NOM_USUEL']].drop_duplicates().set_index('NOM_USUEL')

# réorganise le dataframe df_pm (intermédiaire ayant permis le cumul mensuel poste par poste) avec les postes en lignes et une seule colonne 'RR'
df_p= df_agr.loc[param_RR].stack().reset_index(name= param_RR)
# ajoute à df_p une colonne 'NUM_POSTE' avec le numéro du poste
df_p= df_p.join(df_desc_postes, on='NOM_USUEL')

# Période par défaut if start_date= None ou  end_date= None
# if start_date is None:
#     start_date= df_p.reset_index()['DATE'].min()
# if end_date is None:
#     end_date= df_p.reset_index()['DATE'].max()

# filtre le dataframe sur les postes choisis 
df_p= df_p[df_p['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_p= df_p[(df_p['DATE'] >= start_date) & (df_p['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
# color_sequence_scatter = plotly.colors.qualitative.Dark24

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_p[df_p['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_p[df_p['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param_RR], 
                          # name= poste
                         name= df_p[df_p['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param_RR].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'),
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                             mode='lines' 
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations mensuelles (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                  )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot

# Définition générale des axes
max_value= df_p[param_RR].max()
max_value_cum= df_p.groupby(['NUM_POSTE']).sum()[param_RR].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# Graduations de l'axe des DATES ...............................................................
if end_date - start_date > datetime.timedelta(days= 365): # Plus d'un an
    tickformat= '%b\n%Y'
    dtick= 'M6'
    showgrid_x= True
else: # Moins d'un an
    tickformat= '%b %Y'
    dtick= 'M1'
    showgrid_x= False
# ...............................Premiers subplots
# ................................................ en fonction de la longueur de la période
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ...............................Dernier subplot
# ................................................ en fonction de la longueur de la période
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x')
#                                                SANS tenir compte de la longueur de la période
# fig.update_xaxes(tickformat= '%b\n%Y', tickmode= 'linear', dtick= 'M1', row=len(postes), col=1, matches='x')

# GRILLE .............................................................................
# .................. verticales...
fig.update_xaxes(showgrid= showgrid_x, gridwidth= 1, gridcolor= 'white')
# .................. horizontales
#                                Axe secondaire:
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph_agr= "meteo.data MENS"
fig.write_html(os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".png"))

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
fig.show()
# afficher RR pour df_p avec les postes en colonnes et les dates en index
print('Précipitations mensuelles LATEST (mm)')
display(df_p.pivot(index= 'DATE', columns= 'NOM_USUEL', values= param_RR))

2023-12-31 14:48


Précipitations mensuelles LATEST (mm)


NOM_USUEL,AIX EN PROVENCE,APT-VITON,CASTELLANE,FORCALQUIER,FREJUS,LA BASTIDE DES JOURDANS,LE LUC,MARSEILLE-OBS,SALON DE PROVENCE,TOULON,VALENSOLE,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-09-01,26.4,16.8,60.1,108.0,29.9,41.7,32.6,46.2,17.4,27.1,32.6,54.5
2023-10-01,47.5,84.1,151.1,77.5,83.1,67.4,72.0,30.3,50.1,115.0,91.1,86.0
2023-11-01,30.2,37.6,102.2,63.1,20.4,22.0,21.7,23.2,30.8,25.8,36.4,41.8
2023-12-01,42.6,60.4,76.3,NaN,50.6,42.6,53.8,28.3,35.4,65.4,56.8,71.0


##### 4) Sauvegarde les données intégrales dans un fichier Excel (tous les paramètres pour tous les postes des départements concernés)
Le fichier est sauvegardé dans le même dossier local que les fichiers CSV décompressés

In [12]:
# Sauvegarde toutes les informations dans un fichier Excel
print(now.strftime("%Y-%m-%d %H:%M"))
print('Sauvegarde en cours, patientez...')
file_excel= os.path.join(folder_csv, "meteo.data QUOT"+ '_' + '-'.join(departements) + template_end[:-4] + ".xlsx")
writer = pd.ExcelWriter(file_excel, engine='xlsxwriter')
workbook  = writer.book
#======================== Sauvegarde la liste des postes dans le fichier excel
# sauvegarde dans un fichier excel plus lisible que le JSON après suppression des colonnes type, geometry et properties
sheet= 'liste_postes'
df_liste_postes.to_excel(writer, sheet_name= sheet, index= False, startrow= 0)

#======================== Sauvegarde la liste des champs dans le fichier Excel
sheet= 'Q_descriptif_champs_RR-T-Vent'
df_desc_f.to_excel(writer, sheet_name= sheet, index= True, startrow= 1)
worksheet = writer.sheets[sheet]
worksheet.write('A1', sheet + '.csv')

#======================== Sauvegarde les données complètes dans le fichier Excel    
sheet= 'donnée complètes'
df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', 'Données LATEST quotidiennes au format original, complètes pour tous les départements concernés: ')

#======================== Sauvegarde les données sélectionnées dans le fichier Excel
n_param= len(param_list)
sheet= 'QUOT Sélection'
df_f[(df_f.index.get_level_values(1) >= start_date) & (df_f.index.get_level_values(1) <= end_date)] \
    .to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_f)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_f.loc[par]['name_long']
    worksheet.write('A' + str(i+3), titre)

sheet= 'MENS Sélection'
df_agr[(df_agr.index.get_level_values(1) >= start_date) & (df_agr.index.get_level_values(1) <= end_date)] \
    .to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_agr)
for i, par in enumerate(param_list):
    if par in df_desc_bis.index:
        titre= '- ' + par + ': ' + df_desc_bis.loc[par]['name_long']
    else:
        titre= '- ' + par + ': ' + 'agrégation mensuelle "' + agregat_list[i] + '" de "' + df_desc_f.loc[par]['name_long'] + '"'
    worksheet.write('A' + str(i+3), titre)

# ajoute un onglet excel et enregistre une image du graphique
sheet4= 'graph'
worksheet = workbook.add_worksheet(sheet4)
worksheet.insert_image('A1', os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))
worksheet.insert_image('Q1', os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".png"))

writer.save()

print('Ficher Excel sauvegardé: ', file_excel)

2023-12-31 13:43
Sauvegarde en cours, patientez...
Ficher Excel sauvegardé:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\Q\meteo.data QUOT_04-13-83-84_latest-2022-2023_RR-T-Vent.xlsx
